In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

24/04/06 10:12:51 WARN Utils: Your hostname, Luiss-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.100.206 instead (on interface en1)
24/04/06 10:12:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/06 10:12:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.ml.regression import LinearRegression

In [9]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [10]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [13]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [16]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [18]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App','Time on Website','Length of Membership'], outputCol='features')

In [19]:
output = assembler.transform(data)

In [20]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [21]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [22]:
final_data = output.select('features', 'Yearly Amount Spent')

In [23]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [24]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [26]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                347|
|   mean| 500.08538879318957|
| stddev|  79.34481023217988|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [27]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                153|
|   mean| 497.56463541214845|
| stddev|  79.47916505476778|
|    min|  275.9184206503857|
|    max|  725.5848140556806|
+-------+-------------------+



In [28]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [29]:
lr_model = lr.fit(train_data)

24/04/06 10:22:07 WARN Instrumentation: [863e9318] regParam is zero, which might cause numerical instability and overfitting.
24/04/06 10:22:08 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/06 10:22:08 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [30]:
test_results = lr_model.evaluate(test_data)

In [31]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-11.855371733753259|
|-0.6451327242237426|
| 10.536025548113344|
| -3.958039372588189|
|-6.3543862684505825|
| -21.60234930320769|
|  2.824372281587671|
| -4.177766416235727|
|  2.484442070653756|
| -8.876276425156334|
| -6.354472707309469|
| -5.139393721715635|
|  3.424049073315757|
|-14.529001817290691|
| -2.349925180032301|
| 17.889517544503406|
|-1.2379638593994287|
|   -8.0571517878833|
|-1.7965499995115692|
| 1.8975364851301606|
+-------------------+
only showing top 20 rows



In [32]:
test_results.rootMeanSquaredError

8.98622656280656

In [33]:
test_results.r2

0.9871324466214542

In [34]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [39]:
unlabeled_data = test_data.select('features')

In [41]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.5743636841713...|
|[30.7377203726281...|
|[30.8794843441274...|
|[31.0613251567161...|
|[31.1239743499119...|
|[31.3091926408918...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5316044825729...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6610498227460...|
|[31.7207699002873...|
|[31.7216523605090...|
|[31.7366356860502...|
+--------------------+
only showing top 20 rows



In [42]:
predictions = lr_model.transform(unlabeled_data)

In [43]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|331.78424153694687|
|[30.5743636841713...| 442.7095464822894|
|[30.7377203726281...|451.24471664811654|
|[30.8794843441274...|494.16463935744287|
|[31.0613251567161...| 493.9098443263522|
|[31.1239743499119...|508.54940314297346|
|[31.3091926408918...|429.89634555834596|
|[31.4252268808548...| 534.9444850709976|
|[31.4459724827577...| 482.3925228644748|
|[31.4474464941278...|427.47901852038035|
|[31.5147378578019...| 496.1669607037709|
|[31.5171218025062...|281.05781437210135|
|[31.5316044825729...| 433.0915566560468|
|[31.5741380228732...| 558.9382739778775|
|[31.5761319713222...| 543.5765091693606|
|[31.6005122003032...|461.28333394659353|
|[31.6610498227460...|417.59631743930026|
|[31.7207699002873...| 546.8320852659062|
|[31.7216523605090...| 349.5734766313842|
|[31.7366356860502...|495.03590977040176|
+--------------------+------------